## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [20]:
import random
import statistics

# генерируем массив A с N=1млн случайных целых чисел на отрезке от 0 до 1000
N = 1000000
A = [random.randint(0, 1000) for _ in range(N)]

# создаем массив B
B = [num + 100 for num in A]

# вычисляем среднее значение массива B
mean_B = statistics.mean(B)

print(mean_B)

599.864634


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [21]:
import pandas as pd
import numpy as np
import string

# создаем таблицу с 2млн строк и 4 столбцами со случайными числами
N = 2000000
df = pd.DataFrame(np.random.randint(0, 1000, size=(N, 4)), columns=list('ABCD'))

# добавляем столбец key с случайными выборками из первых пяти букв английского алфавита
df['key'] = np.random.choice(list(string.ascii_uppercase)[:5], N)

print(df.head())

     A    B    C    D key
0  323  712   62  886   B
1  888  328   34   97   E
2  503  481   59  170   C
3  920   90   88  158   C
4  716  167  332  161   C


## Лабораторная работа 3

In [1]:
import pandas as pd


In [ ]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [5]:
recipes = pd.read_csv('recipes_sample.csv').dropna()
reviews = pd.read_csv('reviews_sample.csv').dropna()

In [11]:
print(recipes.dtypes)
print('')
print(reviews.dtypes)

name                      object
id                         int64
minutes                    int64
contributor_id             int64
submitted         datetime64[ns]
n_steps                  float64
description               object
n_ingredients            float64
dtype: object

Unnamed: 0             int64
user_id                int64
recipe_id              int64
date          datetime64[ns]
rating                 int64
review                object
dtype: object


In [10]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
reviews['date'] = pd.to_datetime(reviews['date'])

In [15]:
import pandas as pd
import time


def calculate_mean_rating_a(reviews):
    count = 0
    ratings_sum = 0
    for _, row in reviews.iterrows():
        if pd.to_datetime(row['date']).year == 2010:
            ratings_sum += row['rating']
            count += 1
    return ratings_sum / count


def calculate_mean_rating_b(reviews):
    count = 0
    ratings_sum = 0
    for _, row in reviews[reviews['date'].dt.year == 2010].iterrows():
        ratings_sum += row['rating']
        count += 1
    return ratings_sum / count


def calculate_mean_rating_c(reviews):
    return reviews[reviews['date'].dt.year == 2010]['rating'].mean()


recipes = pd.read_csv('recipes_sample.csv', index_col=0)
reviews = pd.read_csv('reviews_sample.csv', index_col=0)

reviews['rating'] = reviews['rating'].astype(float)
reviews['date'] = pd.to_datetime(reviews['date'])


start_time = time.time()
mean_rating_a = calculate_mean_rating_a(reviews)
end_time = time.time()
print(f"Средний рейтинг (метод A): {mean_rating_a} Время выполнения: {end_time-start_time}")

start_time = time.time()
mean_rating_b = calculate_mean_rating_b(reviews)
end_time = time.time()
print(f"Средний рейтинг (метод B): {mean_rating_b} Время выполнения: {end_time-start_time}")

start_time = time.time()
mean_rating_c = calculate_mean_rating_c(reviews)
end_time = time.time()
print(f"Средний рейтинг (метод C): {mean_rating_c} Время выполнения: {end_time-start_time}")

Средний рейтинг (метод A): 4.4544402182900615 Время выполнения: 6.360488176345825
Средний рейтинг (метод B): 4.4544402182900615 Время выполнения: 0.8165831565856934
Средний рейтинг (метод C): 4.4544402182900615 Время выполнения: 0.026965856552124023


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f calculate_mean_rating_b calculate_mean_rating_b(reviews)

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [12]:
%%time

def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 25.3 µs


In [14]:
%%time

from collections import defaultdict

def get_word_reviews_count(df):
    word_reviews = defaultdict(list)   #defaultdict никогда не вызывает никаких KeyError, поскольку он предоставляет значение по умолчанию для ключа, которого нет в словаре, созданном пользователем
    word_reviews_count = defaultdict(int)
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split()
        unique_words = set(words) 
        for word in unique_words:
            word_reviews[word].append(recipe_id)
            word_reviews_count[word] += 1
        
    return dict(word_reviews_count)


CPU times: user 12 µs, sys: 1 µs, total: 13 µs
Wall time: 17.6 µs


Основные изменения включают в себя:

  * Использование defaultdict из модуля collections, чтобы избежать проверки 
наличия ключа в словаре перед доступом к нему.

  * Вычисление уникальных слов в каждом отзыве с использованием набора, чтобы избежать повторного подсчета одного и того же рецепта.

  * Использование одного цикла для обновления словарей word_reviews и word_reviews_count с сохранением одной итерации по фрейму данных

  * удалилa аргумент в методе split(), поскольку по умолчанию он разбивается на пробелы.

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [19]:
import pandas as pd
import numpy as np
from numba import jit

# удаляем отзывы с нулевым рейтингом.
reviews = reviews[reviews['rating'] > 0]

def mape(ratings):
    """
    Calculate the MAPE for the given ratings.
    """
    mean_rating = np.mean(ratings)
    abs_pct_error = np.abs(ratings - mean_rating) / mean_rating
    mape = np.mean(abs_pct_error) * 100
    return mape

# Версия 1: Без использования векторизованных операций и методов массивов numpy и без использования numba
def mape_v1(reviews):
    mape_values = []
    for recipe_id in reviews['recipe_id'].unique():
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating']
        mape_ = mape(ratings)
        mape_values.append(mape_)
    return mape_values

# Версия 2: Без использования векторизованных операций и методов массивов numpy, но с использованием numba
@jit(nopython=True)
def mape_numba(ratings, mean_rating):
    """
    Calculate the MAPE for the given ratings using numba to speed up the
    calculations.
    """
    n = ratings.shape[0]
    abs_pct_error = np.empty(n)
    for i in range(n):
        abs_pct_error[i] = abs(ratings[i] - mean_rating) / mean_rating
    mape = np.mean(abs_pct_error) * 100
    return mape

def mape_v2(reviews):
    mape_values = []
    for recipe_id in reviews['recipe_id'].unique():
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating']
        mean_rating = np.mean(ratings)
        mape_ = mape_numba(ratings.values, mean_rating)
        mape_values.append(mape_)
    return mape_values

                                   
# Версия 3: С использованием векторизованных операций и методов массивов numpy, но без использования numba
def mape_v3(reviews):
    recipe_ids = reviews['recipe_id'].unique()
    mean_ratings = reviews.groupby('recipe_id')['rating'].transform('mean')
    mape_values = []
    for i, recipe_id in enumerate(recipe_ids):
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating']
        mean_rating = mean_ratings[i]
        abs_pct_error = np.abs(ratings - mean_rating) / mean_rating
        mape = np.mean(abs_pct_error) * 100
        mape_values.append(mape)
    return mape_values

# Версия 4: C использованием векторизованных операций и методов массивов numpy и numba
@jit(nopython=True)
def mape_numba_v2(ratings, mean_rating):
    """
    Calculate the MAPE for the given ratings using vectorization and numba to
    speed up the calculations.
    """
    abs_pct_error = np.abs(ratings - mean_rating) / mean_rating
    mape = np.mean(abs_pct_error) * 100
    return mape

def mape_v4(reviews):
    recipe_ids = reviews['recipe_id'].unique()
    mean_ratings = reviews.groupby('recipe_id')['rating'].transform('mean').values
    mape_values = np.empty(len(recipe_ids))
    for i, recipe_id in enumerate(recipe_ids):
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating'].values
        mean_rating = mean_ratings[i]
        mape = mape_numba_v2(ratings, mean_rating)
        mape_values[i] = mape
    return mape_values

